In [1]:
from util.sampler import Sampler
from util.pvalranges_calculator import PvalueCalculator
from util.utils import scan_write_metrics, customsort
from tqdm import tqdm
from multiprocessing import Pool
import pickle
from pathlib import Path

In [2]:
upstream = ["load"]
product = None
scoring = None
typerun = None

In [3]:
# Parameters
scoring = "hc"
typerun = "group"
upstream = {"load": {"abnormal": "/Users/celiacintas/Code/PDSS_private/output/molformer/bbbp_finetune/group/hc/abnormal.pkl", "bg": "/Users/celiacintas/Code/PDSS_private/output/molformer/bbbp_finetune/group/hc/bg.pkl", "clean": "/Users/celiacintas/Code/PDSS_private/output/molformer/bbbp_finetune/group/hc/clean.pkl", "nb": "/Users/celiacintas/Code/PDSS_private/output/molformer/bbbp_finetune/group/hc/load.ipynb"}}
product = {"nb": "/Users/celiacintas/Code/PDSS_private/output/molformer/bbbp_finetune/group/hc/runs_ss.ipynb", "clean_output": "/Users/celiacintas/Code/PDSS_private/output/molformer/bbbp_finetune/group/hc/clean_output.out", "adv_output": "/Users/celiacintas/Code/PDSS_private/output/molformer/bbbp_finetune/group/hc/adv_output.out", "indices": "/Users/celiacintas/Code/PDSS_private/output/molformer/bbbp_finetune/group/hc/indices.pkl"}


In [4]:
bg = pickle.loads(Path(upstream["load"]["bg"]).read_bytes())
abnormal = pickle.loads(Path(upstream["load"]["abnormal"]).read_bytes())
clean = pickle.loads(Path(upstream["load"]["clean"]).read_bytes())
print(bg.shape, clean.shape, abnormal.shape)

(1167, 768) (400, 768) (483, 768)


## Run subset scanning

In [5]:
runs = {
    "group": {
        "clean": {"clean_ssize": 200, "anom_ssize": 0},
        "abnormal": {"clean_ssize": 250, "anom_ssize": 50},
    },
    "individual": {
        "clean": {"clean_ssize": 1, "anom_ssize": 0},
        "abnormal": {"clean_ssize": 0, "anom_ssize": 1},
    },
}

run = 100

for key in ["clean", "abnormal"]:
    run_indices = list()
    print("Run for key: {}".format(key))
    clean_ssize = runs[typerun][key]["clean_ssize"]
    anom_ssize = runs[typerun][key]["anom_ssize"]
    if (clean_ssize != 1 and typerun == "group") or (
        clean_ssize == 1 and typerun == "individual"
    ):
        resultsfile = Path(product["clean_output"])
    if anom_ssize != 0:
        resultsfile = Path(product["adv_output"])

    bg = customsort(bg, conditional=False)
    pvalcalculator = PvalueCalculator(bg)

    records_pvalue_ranges = pvalcalculator.get_pvalue_ranges(clean, pvaltest="1tail")
    anom_records_pvalue_ranges = pvalcalculator.get_pvalue_ranges(
        abnormal, pvaltest="1tail"
    )

    if anom_ssize == 1 and clean_ssize == 0:
        run = anom_records_pvalue_ranges.shape[0]

    elif clean_ssize == 1 and anom_ssize == 0:
        run = records_pvalue_ranges.shape[0]

    samples, sampled_indices = Sampler.sample(
        records_pvalue_ranges,
        anom_records_pvalue_ranges,
        clean_ssize,
        anom_ssize,
        run,
        conditional=False,
    )
    run_indices.append(sampled_indices)

    pool = Pool(processes=5)
    calls = []

    for r_indx in range(run):
        pred_classes = None
        run_sampled_indices = None
        sampled_indices = None

        calls.append(
            pool.apply_async(
                scan_write_metrics,
                [
                    samples[r_indx],
                    pred_classes,
                    clean_ssize,
                    anom_ssize,
                    resultsfile,
                    1,
                    False,
                    None,
                    scoring,
                    -1,
                    run_sampled_indices,
                ],
            )
        )

    print("Beginning Scanning...")
    for sample in tqdm(calls):
        sample.get()

Run for key: clean


Beginning Scanning...


  0%|                                                                                                                                                | 0/100 [00:00<?, ?it/s]

  1%|█▎                                                                                                                                      | 1/100 [00:00<00:13,  7.40it/s]

 13%|█████████████████▌                                                                                                                     | 13/100 [00:00<00:01, 56.60it/s]

...

 72%|█████████████████████████████████████████████████████████████████████████████████████████████████▏                                     | 72/100 [00:01<00:00, 54.48it/s]

 80%|████████████████████████████████████████████████████████████████████████████████████████████████████████████                           | 80/100 [00:01<00:00, 59.68it/s]

 87%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                 | 87/100 [00:01<00:00, 59.07it/s]

 95%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎      | 95/100 [00:01<00:00, 58.09it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 55.66it/s]

In [6]:
Path("output").mkdir(exist_ok=True)
Path(product["indices"]).parent.mkdir(exist_ok=True, parents=True)
Path(product["indices"]).write_bytes(pickle.dumps(run_indices))

246952